In [ ]:
import time
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json
from activity_detector import ActivityDetector, LateralActivityHost, LateralActivityTarget, LeadVehicle
from ngram import NGram
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [ ]:
i_file = 0
datafiles = glob(os.path.join("data", "1_hdf5", '*.hdf5'))
df = pd.read_hdf(datafiles[i_file])
AD = ActivityDetector(df)

## Create or load n-grams for target vehicles

In [ ]:
filename = os.path.join("data", "4_json", "{:s}_targets.json".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["longitudinal_activity", "lateral_activity", "longitudinal_state", 
              "lateral_state", "lead_vehicle"]
metadata = (("tstart", float), ("tend", float), ("target_id", int), ("target_tracker", int))
target_ngrams = NGram(fieldnames, metadata)

In [ ]:
if os.path.exists(filename):
    target_ngrams.from_json(filename)
else:
    for i_target in range(8):
        t_field_names = [AD.target_signal(i_target, fieldname) for fieldname in fieldnames]
        mapper = dict()
        for oldname, newname in zip(t_field_names, fieldnames):
            mapper[oldname] = newname
        dftargets = df[t_field_names].rename(columns=mapper)
        targetids = AD.get_t(i_target, "id")
        ids = np.unique(targetids)

        for targetid in ids[1:]:  # Skip the first one, which is always 0.
            dftarget = dftargets.loc[targetids == targetid]
            target_ngrams.ngram_from_data(dftarget, tstart=dftarget.index[0], tend=dftarget.index[-1],
                                          target_id=int(targetid), target_tracker=i_target)
    target_ngrams.sort_ngrams("tstart")
    target_ngrams.to_json(filename)

## Create or load n-gram for ego vehicle

In [ ]:
filename = os.path.join("data", "4_json", "{:s}_ego.json".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["host_longitudinal_activity", "host_lateral_activity",
              "is_highway"]
metadata = (("tstart", float), ("tend", float))
ego_ngram = NGram(fieldnames, metadata)

In [ ]:
if os.path.exists(filename):
    ego_ngram.from_json(filename)
else:
    ego_ngram.ngram_from_data(df, tstart=df.index[0], tend=df.index[-1])
    ego_ngram.to_json(filename)

## Extract cut-in scenario

For target vehicle, we need:
1. Lateral activity `li` or `ri`.
2. Lateral activity `fl` and lead vehicle `y`.

For the ego vehicle, we need it to go straight during step 2 of the target vehicle.

In [ ]:
def check_row(row, dict_tags: dict):
    """ Check if a row of a dataframe contains the provided tags. 
    
    Each item of the dictionary needs to contain a list. 
    
    :param row: The row that is obtained through pd.DataFrame.itertuples().
    :param dict_tags: The dictionary of tags.
    """
    for key, tags in dict_tags.items():
        if getattr(row, key) not in tags:
            return False
    return True

In [ ]:
target_tags = [dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value]),
               dict(lateral_activity=[LateralActivityTarget.LANE_FOLLOWING.value], 
                    lead_vehicle=[LeadVehicle.LEAD.value])]
ego_tags = [dict(lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True])]

In [ ]:
i_start, i_step = 0, 0

ngram = target_ngrams.ngrams[365]
for row in ngram.data.itertuples():
    if i_step < len(target_tags) and check_row(row, target_tags[i_step]):
        print("Step {:d} at time {:.2f}".format(i_step+1, row.Index))

In [ ]:
# Tracker id is 1 and 4.
tstart = 2466
tend = 2466.5
for i in range(8):
    plt.plot(df.loc[tstart:tend, "Target_{:d}_dy".format(i)],
             df.loc[tstart:tend, "Target_{:d}_dx".format(i)], '.')
plt.xlim(-5, 5)
plt.ylim(0, 50)

In [ ]:
target_ngrams.ngrams[4000].meta["tstart"]

In [ ]:
df.loc[tstart:tend, ["Target_1_dx", "Target_4_dx", "Target_1_id", "Target_4_id"]]

In [ ]:
ego_ngram.ngrams[0].meta

In [ ]:
plt.plot(df.loc[2870:, "Host_vx"])

In [ ]:
47*60+46

In [ ]:
47*60+55

In [ ]:
# lon_events = AD.lon_activities_host()
lat_events = AD.lat_activities_host()

In [ ]:
df.iloc[1:4]["line_left_y_conf"]

In [ ]:
tmp = np.zeros(len(df))
for i in range(len(df)):
    istart = max(0, i - 100)
    iend = i+1
    tmp[i] = np.max(df.iloc[istart:iend]["line_left_y_conf"])

In [ ]:
#plt.plot(df["line_left_y_conf_down"])
plt.plot(df["line_left_y_conf"] - tmp - df["line_left_y_conf_down"])

In [ ]:
x = pd.DataFrame([0, 1, 2, 3, 4, 5, 4, 3, 2, 8, 2, 3, 10])

In [ ]:
x.rolling(window=3).max()

In [ ]:
n = 10000
tmp = np.zeros(n)
for i in range(n):
    istart = max(0, i - 100+1)
    iend = i+1
    tmp[i] = np.max(df.iloc[istart:iend]["line_left_y_conf"])

In [ ]:
plt.plot(df["lines_0_c0"].iloc[10000:30000])
i = 203.05
plt.plot(i, df.at[i, "line_left_y_conf"], "r.")

In [ ]:
lat_events

In [ ]:
def find_neighbours(value):
    exactmatch = df[df.video_time == value]
    if not exactmatch.empty:
        return exactmatch.index[0]
    lowerneighbour_ind = df[df.video_time < value].idxmax()
    return lowerneighbour_ind

def approx_index(minute, second):
    iframe = (minute*60 + second)*10
    print("Seconds: {:.0f}".format(minute*60+second))
    print("Index: {:.2f}".format(find_neighbours(iframe)))
    return find_neighbours(iframe)
          
_ = approx_index(44,23)

In [ ]:
i = approx_index(37,5)
istart = i - 10
iend = i + 10
plt.plot(df.loc[istart:iend, "lines_0_c0"], color=[.8, .8, 1])
plt.plot(df.loc[istart:iend, "lines_1_c0"], color=[1, .8, .8])
plt.plot(df.loc[istart:iend, "line_left_y_conf"], color=[0, 0, 1])
plt.plot(df.loc[istart:iend, "line_right_y_conf"], color=[1, 0, 0])
plt.plot([istart, iend], [0, 0], 'k')

In [ ]:
plt.plot(df.loc[istart:iend, "lines_0_quality"])
plt.plot(df.loc[istart:iend, "lines_1_quality"])

In [ ]:
left_y = df["lines_0_c0"]
left_dy = df["line_left_y_conf_down"]
right_y = df["lines_0_c1"]
right_dy = df["line_right_y_conf_up"]

In [ ]:
previous_y = (left_y.iloc[0], right_y.iloc[1])
lane_changes = 0
for row in df.itertuples():
    # Left lane change (out of ego lane).
    lane_change = True
    if row.lines_0_c0 >= 0:
        lane_change = False
    if previous_y[0] < 0:
        lane_change = False
    if np.isnan(row.line_left_y_conf_down):
        lane_change = False
    if np.abs(previous_y[0] - row.lines_0_c0) >= 1:
        lane_change = False
    previous_y = (row.lines_0_c0, row.lines_0_c1)
    if lane_change:
        print("Potential left lane change at i={:.2f}".format(row.Index))
        lane_changes += 1
        if lane_changes >= 20:
            break
        
        # Find start of the lane change.